In [117]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from mne.stats import permutation_cluster_test
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization

### Add Directories/ Load Files

In [118]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


### Select Peaks and Plot Analytic Signal

In [159]:
#Load raw fif data

test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'without_med_FTG',
        'Sub043_FIF.fif'
    )
)

subID = 'Sub043'
fft_name = str(subID) + '_'
print(fft_name)

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\without_med_FTG\Sub043_FIF.fif...
    Reading extended channel information
    Range : 0 ... 129468 =      0.000 ...   517.872 secs
Ready.
Sub043_


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_13440\504685141.py:3: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\without_med_FTG\Sub043_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  test_raw = mne.io.read_raw_fif(os.path.join(


In [163]:
reload(dat_preproc)
x = test_raw.get_data() 
x1 = x[1,:]

#peakMed = 82
peakStim = 62

#dat_ngam = dat_preproc.low_highpass_filter(x1, peakMed-2, peakMed+2) 
dat_subh = dat_preproc.low_highpass_filter(x1, peakStim-2, peakStim+2) 

#datall = [dat_ngam, dat_subh] 
#labels = ['Peak'+str(peakMed)+'Hz','Peak'+str(peakStim)+'Hz']

datall = [dat_subh] 
labels = ['Peak'+str(peakStim)+'Hz']

print(labels)

['Peak62Hz']


In [161]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [164]:
%matplotlib qt
sm_signal_np = np.empty(shape = (1, x1.shape[0] - 499))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(1, 1, figsize=(12, 5))
wintosmooth = 500

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(np.squeeze(amplitude_envelope))

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    plt.plot(sm_signal, label = labels[idx])
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    #axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    #axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    plt.ylabel('Analytic Signal')
    plt.xlim([0, sm_signal.shape[0]])

    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 500 samples')
plt.legend(loc='upper right')

plt.show()


In [167]:
x2 = x[5, :] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim)/2

In [168]:
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,250:], label = labels[idx], lw = 2)
ax2.plot(sm_stim1[250:], label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
#ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
#ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
#ax1.set_xlim(0, sm_signal_np.shape[0])
#ax1.set_xticks(np.arange(0, 100000, 20000))
#ax1.set_xticklabels(np.arange(0, 400, 80))
ax1.set_xlabel('Time [samples]')
plt.title(str(subID))
plt.show()

In [169]:
#sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]], [sm_signal_np[1]],[sm_stim1]])))
sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]],[sm_stim1]])))
sm_analSignal_df = pd.DataFrame(sm_analSignal, 
    columns = ['StimOn','StimVec'],
    )
print(sm_analSignal_df)

           StimOn  StimVec
0       15.966395      0.0
1       15.894239      0.0
2       15.600586      0.0
3       15.213439      0.0
4       14.908738      0.0
...           ...      ...
128965   0.310202      NaN
128966   0.380156      NaN
128967   0.379803      NaN
128968   0.556270      NaN
128969   0.556000      NaN

[128970 rows x 2 columns]


In [170]:
fft_fig = os.path.join(project_path, 'figures','anal_signal','without_med_FTG/')
fft_file = os.path.join(project_path, 'data','anal_signal','without_med_FTG/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignal',dpi = 300)
sm_analSignal_df.to_csv(str(fft_file)+str(fft_name)+'sm_analSignal.csv')

### Crop Analytic Signal to Epochs of Interest

In [178]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'without_med_FTG\\'
        'Sub050_sm_analSignal.csv'
))

print(sm_anal)

subID = 'Sub050'
fft_name = str(subID) + '_'
print(fft_name)

        Unnamed: 0    StimOn  StimVec
0                0  0.754467      0.0
1                1  0.755404      0.0
2                2  0.755474      0.0
3                3  0.754679      0.0
4                4  0.754767      0.0
...            ...       ...      ...
202746      202746  0.649214      NaN
202747      202747  0.651784      NaN
202748      202748  0.654198      NaN
202749      202749  0.655498      NaN
202750      202750  0.658021      NaN

[202751 rows x 3 columns]
Sub050_


In [ ]:
300000/250

In [ ]:
np.arange(0,1400,200)

In [179]:
%matplotlib qt
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
#ax1.plot(np.arange(0, sm_anal.shape[0]), sm_anal['Spontan'], label = 'Peak 81Hz')
ax1.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimOn'], label = 'Peak 65Hz')
ax2.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimVec'], ls='--', color = 'grey')

'''
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax1.set_xticklabels(np.arange(-200,1400,200))
ax1.set_xlabel('Time [sec]')
'''


"\nax1.legend()\nax1.set_ylabel('Z-scored Smoothed Analytic Signal')\nax2.set_ylabel('Stimulation Amplitude [mA]')\nax1.set_xticklabels(np.arange(-200,1400,200))\nax1.set_xlabel('Time [sec]')\n"

In [ ]:
20*250

In [180]:
#CROP IT

#med1 = 52
#med2 = 82
sub_pt = 124575
time_on = sub_pt - 15*250
time_off = sub_pt + 15*250

#sm_anal1 = sm_anal.iloc[med1*250 : med2*250]
sm_anal2 = sm_anal.iloc[time_on : time_off]
sm_toplot = pd.concat([sm_anal2])
print(sm_toplot)

        Unnamed: 0    StimOn  StimVec
120825      120825  0.384783      2.4
120826      120826  0.384579      2.4
120827      120827  0.384384      2.4
120828      120828  0.384200      2.4
120829      120829  0.384026      2.4
...            ...       ...      ...
128320      128320  0.819225      2.5
128321      128321  0.818297      2.5
128322      128322  0.817270      2.5
128323      128323  0.816153      2.5
128324      128324  0.814953      2.5

[7500 rows x 3 columns]


In [181]:
# CROP ONLY THE TRANSITION E.G. BETWEEN 0.5 - 1.0mA
this_sig = sm_toplot

fig, ax1 = plt.subplots(figsize = (8,6))
ax2 = ax1.twinx()

#ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['Spontan'], label = 'Med-Induced')
ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['StimOn'], label = 'Subharmonic')
ax2.plot(np.arange(0, this_sig.shape[0]), this_sig['StimVec'], ls='--', color = 'grey')
ax1.axvline(x = 3750, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
#ax1.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)


ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax1.set_xlabel('Time [samples]')
ax1.set_title(str(subID))


Text(0.5, 1.0, 'Sub050')

In [182]:
fft_fig = os.path.join(project_path, 'figures','anal_signal', 'without_med_FTG/')
fft_file = os.path.join(project_path, 'data','anal_signal', 'without_med_FTG/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalTRANSITION',dpi = 150)
this_sig.to_csv(str(fft_file)+str(fft_name)+'sm_analSignalTRANSITION.csv')

### Average Cropped Analytic Signal and Plot

In [ ]:
'''
med_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllMed_analSignalTRANSITION.csv'
))

stim_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllStim_analSignalTRANSITION.csv'
))

print(stim_anal)
'''

med_anal = []
stim_anal = []

ps_path = os.path.join(
   project_path,
   'data', 'anal_signal'
)

for filename in os.listdir(ps_path):
   if filename.endswith('TRANSITION.csv'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
      this_df = pd.read_csv(os.path.join(ps_path,filename))
      this_med_anal = this_df['Spontan']
      this_stim_anal = this_df['StimOn']


      med_anal.append(this_med_anal)
      stim_anal.append(this_stim_anal)



In [ ]:
all_med_anal_df = pd.DataFrame(med_anal)
all_stim_anal_df = pd.DataFrame(stim_anal)
all_med_anal_df.shape

In [ ]:
med_anal_cr = all_med_anal_df.loc[:, 1:7500] #all_med_anal_df.loc[:, 7500:17500]
med_anal_mean = np.nanmean(med_anal_cr,0)
med_anal_sem = stats.sem(med_anal_cr,0, nan_policy='omit')

stim_anal_cr = all_stim_anal_df.loc[:,  1:7500]
stim_anal_mean = np.nanmean(stim_anal_cr,0)
stim_anal_sem = stats.sem(stim_anal_cr,0, nan_policy='omit')


In [ ]:
med_anal_cr


In [ ]:
2000/250

In [ ]:
%matplotlib qt
plt.plot(np.arange(0,7500), med_anal_mean, label = 'Med-Induced FTG') #10000
plt.fill_between(np.arange(0,7500), med_anal_mean-med_anal_sem, med_anal_mean+med_anal_sem, alpha = 0.2)

plt.plot(np.arange(0,7500), stim_anal_mean, color = 'red', label = 'Stim-Induced FTG')
plt.fill_between(np.arange(0,7500), stim_anal_mean-stim_anal_sem, stim_anal_mean+stim_anal_sem, color = 'red', alpha = 0.2)


#plt.axvline(x = 6250, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
#plt.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)
#plt.xlim(0,10000)
#plt.xticks(np.arange(0,11250,1250), labels = np.arange(-25,20,5))
plt.xlim(0,2500)

plt.legend(loc = 'upper left')
plt.xlabel('Time [sec]')
plt.ylabel('Z-scored Smoothed Analytic Signal')

In [ ]:
fft_fig = os.path.join(project_path, 'results\\')

plt.savefig(str(fft_fig)+'AVG_AnalCropped_StimOn',dpi = 250)

In [ ]:
pval = 0.05  # arbitrary
dfn = 2 - 1  # degrees of freedom numerator
dfd = med_anal.shape[0] - 2  # degrees of freedom denominator
thresh = scipy.stats.f.ppf(1 - pval, dfn=dfn, dfd=dfd)  # F distribution
print(thresh)

In [ ]:
perm_nd = np.empty([6,med_anal.shape[0]])
perm_nd[:] = np.nan
perm_nd[0,:] = np.transpose(med_anal)

In [ ]:
y = np.expand_dims(med_anal, axis = 2)
y1 = np.expand_dims(stim_anal, axis = 2)


In [ ]:
#H0 = np.empty([1000,6])
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([y, y1], n_permutations=1000,
                             threshold=None, n_jobs=None)


In [ ]:
cluster_p_values

### Make all Signals same Length

In [ ]:
import traces


In [ ]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'Sub005_sm_analSignal.csv'
))

print(sm_anal)

subID = 'Sub005'
fft_name = str(subID) + '_'
print(fft_name)

In [ ]:
%matplotlib qt
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['Spontan'], label = 'Med Induced')
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimOn'], label = 'Stim Induced')
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimVec'], ls='--', color = 'grey')
plt.legend()

In [ ]:
np.unique(np.round(sm_anal['StimVec'], decimals = 1))

In [ ]:
sm_anal['StimVec'] = np.round(sm_anal['StimVec'], decimals = 1)
this_anal = sm_anal
plt.plot(sm_anal['Spontan'][sm_anal['StimVec'] == 0])
plt.plot(sm_anal['StimOn'][sm_anal['StimVec'] == 0])
plt.plot(sm_anal['StimVec'][sm_anal['StimVec'] == 0])

In [ ]:
amp_of_interest = [0, 0.5, 1]

sm_anal['Spontan'][250:300,sm_anal['StimVec'] == 0]